In [2]:
# need to pip3 install urdf_parser_py
import PyKDL as kdl
# import urdf_parser_py.urdf as urdf
from urdf import treeFromFile, kdl_to_mat, list_to_kdl, joint_kdl_to_list
import numpy as np

In [3]:
(ok, tree) = treeFromFile('../urdf/tilted_hex_arm.urdf')

In [4]:
chain = tree.getChain('map', 'end_effector') 
print(f"  - Number of joints: {chain.getNrOfJoints()}")
print(f"  - Number of segments: {chain.getNrOfSegments()}")   
print(f"  - Name of first segment: {chain.getSegment(0).getName()}")
print(f"  - Name of first joint: {chain.getSegment(0).getJoint().getName()}")

# list all joints
print("List all the joints")
for i in range(chain.getNrOfSegments()):
    inertia = chain.getSegment(i).getInertia()
    cog = inertia.getCOG()
    m = inertia.getMass()
    I = inertia.getRotationalInertia()
    print(f"   - Segment ({chain.getSegment(i).getName()}) : joint ({chain.getSegment(i).getJoint().getName()}), m({m}), I({list(I)})")

  - Number of joints: 6
  - Number of segments: 10
  - Name of first segment: x_link
  - Name of first joint: x_joint
List all the joints
   - Segment (x_link) : joint (x_joint), m(0.0), I([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
   - Segment (y_link) : joint (y_joint), m(0.0), I([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
   - Segment (z_link) : joint (z_joint), m(0.0), I([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
   - Segment (base_link_yaw) : joint (yaw_joint), m(0.0), I([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
   - Segment (base_link_pitch) : joint (pitch_joint), m(0.0), I([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
   - Segment (base_link_roll) : joint (roll_joint), m(0.0), I([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
   - Segment (base_link) : joint (base_joint), m(3.02), I([0.08, 0.0, 0.0, 0.0, 0.08, 0.0, 0.0, 0.0, 0.12])
   - Segment (sim_ft_sensor) : joint (ft_sensor_joint), m(0.15), I([0.0001, 0.0, 0.0, 0.0, 0.0001, 0.0, 0.0, 0.0, 0.0001])
   - Segme

In [5]:
ee_chain =tree.getChain('map','end_effector')
n_joints = ee_chain.getNrOfJoints()
n_segments = ee_chain.getNrOfSegments()
print(f'Joints: {n_joints}, Segments: {n_segments}')
for i in range(n_segments):
    s = ee_chain.getSegment(i)
    j = s.getJoint()
    if j.getTypeName()!='None':
        print(f" - segment {i}-{s.getName()} with joint {j.getName()} of type {j.getTypeName()} ")

Joints: 6, Segments: 10
 - segment 0-x_link with joint x_joint of type TransAxis 
 - segment 1-y_link with joint y_joint of type TransAxis 
 - segment 2-z_link with joint z_joint of type TransAxis 
 - segment 3-base_link_yaw with joint yaw_joint of type RotAxis 
 - segment 4-base_link_pitch with joint pitch_joint of type RotAxis 
 - segment 5-base_link_roll with joint roll_joint of type RotAxis 
 - segment 6-base_link with joint base_joint of type Fixed 
 - segment 7-sim_ft_sensor with joint ft_sensor_joint of type Fixed 
 - segment 8-fixed_arm with joint arm_joint of type Fixed 
 - segment 9-end_effector with joint end_effector_joint of type Fixed 


In [6]:
# Forward kinematics
fk = kdl.ChainFkSolverPos_recursive(ee_chain)
eeFrame = kdl.Frame()
q = [0, 0, 2, 0, 0, np.pi/2] # x,y,z, roll,pitch,yaw

q_Kdl = kdl.JntArray(n_joints)
q_Kdl[0] = q[0] # x
q_Kdl[1] = q[1] # y
q_Kdl[2] = q[2] # z
q_Kdl[3] = q[5] # yaw in radians 
q_Kdl[4] = q[4] # pitch in radians 
q_Kdl[5] = q[3] # roll in  radians
fk.JntToCart(q_Kdl, eeFrame)
pos = eeFrame.p
rot = eeFrame.M
print( f"pos={pos}, \nrot={rot}")

T_w_ee = np.array([[rot[0, 0], rot[0, 1], rot[0, 2], pos[0]], 
                        [rot[1, 0], rot[1, 1], rot[1, 2], pos[1]],
                        [rot[2, 0], rot[2, 1], rot[2, 2], pos[2]],
                        [        0,         0,          0,     1]])

rot_q = kdl.Rotation(rot).GetQuaternion()
ypr = kdl.Rotation(rot).GetEulerZYX()
print(f'\n Tw_ee:\n{T_w_ee.round(2)}')
print(f'\n rot_q={np.array(rot_q).round(2)},  ypr={ypr}')

pos=[ 2.51053e-17,        0.41,        1.96], 
rot=[ 6.12323e-17,          -1,           0;
            1, 6.12323e-17,           0;
            0,           0,           1]

 Tw_ee:
[[ 0.   -1.    0.    0.  ]
 [ 1.    0.    0.    0.41]
 [ 0.    0.    1.    1.96]
 [ 0.    0.    0.    1.  ]]

 rot_q=[0.   0.   0.71 0.71],  ypr=(1.5707963267948966, -0.0, 0.0)


In [7]:
# inverse kinematics
# target pose represented as rotation matrix and translation vector 
# result is represented as x,y,z,yaw,pitch,roll

pos_kdl = kdl.Vector(pos[0], pos[1], pos[2]) #xyz position
rot_kdl = kdl.Rotation(rot) # orientation
target_pose_kdl = kdl.Frame(rot_kdl, pos_kdl)


fk_kdl = kdl.ChainFkSolverPos_recursive(ee_chain)
ik_v_kdl = kdl.ChainIkSolverVel_pinv(ee_chain)

# List of joint angles to lower/upper bound the angles on the IK search
q_min = np.array([-100, -100, -100, -2*np.pi, -2*np.pi, -2*np.pi])
q_max =-1* q_min
mins_kdl = list_to_kdl(q_min)
maxs_kdl = list_to_kdl(q_max)

ik_p_kdl = kdl.ChainIkSolverPos_NR_JL(ee_chain, mins_kdl, maxs_kdl, fk_kdl, ik_v_kdl)

# use the midpoint of the joint limits as the guess
q_guess = (q_min + q_max) / 2.0
q_guess = np.where(np.isnan(q_guess), [0.]*len(q_guess), q_guess)

q_kdl = kdl.JntArray(n_joints)
q_guess_kdl = list_to_kdl(q_guess)
if ik_p_kdl.CartToJnt(q_guess_kdl, target_pose_kdl, q_kdl) >= 0:
    print(f"q_ik = {np.array(joint_kdl_to_list(q_kdl)).round(2)}")
    print(f"q = {joint_kdl_to_list(q_Kdl)}")
    
else:
    print("could not find solution")

q_ik = [-0.    0.    2.    1.57 -0.   -0.  ]
q = [0.0, 0.0, 2.0, 1.5707963267948966, 0.0, 0.0]


In [8]:
# Jacobian
jac_solver = kdl.ChainJntToJacSolver(ee_chain)
J = kdl.Jacobian(len(q))
jac_solver.JntToJac(q_Kdl, J)
J = kdl_to_mat(J)
print(f"J=\n{J.round(2)}")

# Conditional Number (manipulability), the greater the value the worse the motion manipulability
cond = np.linalg.cond(J)
print(f"cond={cond}")

J_inv = np.linalg.pinv(J)


J=
[[ 1.    0.    0.   -0.41 -0.   -0.04]
 [ 0.    1.    0.    0.   -0.04  0.  ]
 [ 0.    0.    1.    0.   -0.41  0.  ]
 [ 0.    0.    0.    0.   -1.    0.  ]
 [ 0.    0.    0.    0.    0.    1.  ]
 [ 0.    0.    0.    1.    0.    0.  ]]
cond=1.5054442492474192


In [9]:
"""Calculates local Jacobian (arm base to EE) from fixed-axis jacobian and FK.
The local jacobian is the mobile-axis Jacobian calculated by
J_M = [R.T, 0_3x3;  0_3x3, R.T]*J_F

Args:
    joint (list): current joint positions

Returns:
    np.mat: local jacobian matrix
"""

rot = T_w_ee[0:3, 0:3]
t1 = np.concatenate((rot.transpose(), np.zeros((3, 3))), axis=1)
t2 = np.concatenate((np.zeros((3, 3)), rot.transpose()), axis=1)
Tinv = np.concatenate((t1,  t2), axis=0)
locJacobian = np.matmul(Tinv, J)
print(locJacobian.round(2))
print('\n')
print(J.round(2))

[[ 0.    1.    0.   -0.   -0.04  0.  ]
 [-1.    0.    0.    0.41  0.    0.04]
 [ 0.    0.    1.    0.   -0.41  0.  ]
 [ 0.    0.    0.    0.    0.    1.  ]
 [ 0.    0.    0.    0.    1.    0.  ]
 [ 0.    0.    0.    1.    0.    0.  ]]


[[ 1.    0.    0.   -0.41 -0.   -0.04]
 [ 0.    1.    0.    0.   -0.04  0.  ]
 [ 0.    0.    1.    0.   -0.41  0.  ]
 [ 0.    0.    0.    0.   -1.    0.  ]
 [ 0.    0.    0.    0.    0.    1.  ]
 [ 0.    0.    0.    1.    0.    0.  ]]


In [10]:
# forward velocity kinematics
end_frame = kdl.FrameVel()
fk_v_kdl = kdl.ChainFkSolverVel_recursive(ee_chain)

q_dot = [0.0, 0.1, 0.0,0.0,0,0.1] #vx, vy, vz, yaw_dot, pitch_dot, roll_dot
qdot_jntarr = list_to_kdl(q_dot)
q_dot_kdl = kdl.JntArrayVel(q_kdl,qdot_jntarr)

fk_v_kdl.JntToCart(q_dot_kdl,end_frame)
V = end_frame.GetTwist()
V = np.array([V[0], V[1], V[2], V[3], V[4], V[5]])
print(f"V={V.round(3)}")


# joint_vel_from_task_vel using pseudo-inverse
# q_dot = J_inv*x_dot
# J_inv = np.linalg.pinv(J)
# q_dot = np.matmul(J_inv, (task_velocity))
print(f"V={np.matmul(J,q_dot).round(3)}")

V=[-0.004  0.1   -0.     0.     0.1    0.   ]
V=[[-0.004  0.1    0.     0.     0.1    0.   ]]


In [11]:
# Inertia matrix
grav_vector = kdl.Vector(0., 0., -9.81)  
dyn_param = kdl.ChainDynParam(ee_chain, grav_vector)
inertia = kdl.JntSpaceInertiaMatrix(n_joints)
dyn_param.JntToMass(q_Kdl, inertia)

M = kdl_to_mat(inertia)
M_inv = np.linalg.inv(M)
print(f"M=\n{M.round(2)} \nM_inv=\n{M_inv.round(2)}")

# Cartesian inertia Hc
Hc = np.linalg.inv(J * M_inv * J.T)
print(f"Hc=\n{Hc.round(2)}")

# gravity torques
G_kdl= kdl.JntArray(n_joints)
tau_g = np.zeros((n_joints))
dyn_param.JntToGravity(q_kdl,G_kdl)
for row in range(n_joints):
    tau_g[row] = G_kdl[row]
print(f"gravity torques tau_g: {tau_g.round(3)}")


#coupling torques
q_dot = [0.0, 0.0, 0.0,0.2,0.2,0.0] #vx, vy, vz, yaw_dot, pitch_dot, roll_dot
qdot_jntarr = list_to_kdl(q_dot)

C_kdl= kdl.JntArray(n_joints)
tau_c = np.zeros((n_joints))
dyn_param.JntToCoriolis(q_kdl,qdot_jntarr,C_kdl)
for row in range(n_joints):
    tau_c[row] = C_kdl[row]
print(f"Coriolis torques: {tau_c.round(3)}")

M=
[[ 3.37  0.    0.   -0.08 -0.   -0.01]
 [ 0.    3.37  0.    0.   -0.01  0.  ]
 [ 0.    0.    3.37  0.   -0.08  0.  ]
 [-0.08  0.    0.    0.15  0.    0.  ]
 [-0.   -0.01 -0.08  0.    0.11  0.  ]
 [-0.01  0.    0.    0.    0.    0.08]] 
M_inv=
[[ 0.3  -0.    0.    0.17  0.    0.05]
 [-0.    0.3   0.   -0.    0.04 -0.  ]
 [ 0.    0.    0.3   0.    0.23  0.  ]
 [ 0.17 -0.    0.    6.96  0.   -0.25]
 [ 0.    0.04  0.23  0.    9.67  0.  ]
 [ 0.05 -0.    0.   -0.25  0.   12.38]]
Hc=
[[ 3.37 -0.    0.    0.    0.12  1.3 ]
 [ 0.    3.37  0.   -0.12  0.   -0.  ]
 [-0.    0.    3.37 -1.3   0.   -0.  ]
 [-0.   -0.12 -1.3   0.61 -0.    0.  ]
 [ 0.12 -0.    0.   -0.    0.09  0.05]
 [ 1.3  -0.    0.    0.    0.05  0.65]]
gravity torques tau_g: [ 0.     0.    33.06  -0.    -0.795 -0.   ]
Coriolis torques: [ 0.001 -0.006  0.001 -0.     0.    -0.002]


In [12]:
# Inverse  Dynamics (requires >v1.4, compile from source )
q = [0.0, 0.0, 1.0,0.0,0.0,0.0]
q_kdl = list_to_kdl(q)
q_dot = [0.0, 0.0, 0.0,0.0,0.0,0.0] #vx, vy, vz, yaw_dot, pitch_dot, roll_dot
qd_kdl = list_to_kdl(q_dot)
q_dd = [0., 0.0, 0.0,0.0,0.0,0.] #vx, vy, vz, yaw_dot, pitch_dot, roll_dot
qdd_kdl = list_to_kdl(q_dd)
wrenches = [kdl.Wrench()]*n_segments
tau_gc = kdl.JntArray(n_joints)

ID_solver = kdl.ChainIdSolver_RNE(ee_chain, grav_vector)
ret = ID_solver.CartToJnt(q_kdl, qd_kdl, qdd_kdl, wrenches, tau_gc)
tau = np.zeros((n_joints))
for i in range(n_joints):
    tau[i] = tau_gc[i]
print(tau.round(3))

[ 0.     0.    33.06   0.    -0.795  0.   ]


In [13]:
# External Wrench estimator, not wrapped in PyKDL
# kdl.ChainExternalWrenchEstimator
kdl.__file__

'/usr/local/lib/python3.8/dist-packages/PyKDL.so'